In [ ]:
#installing mol2vec
!pip install git+https://github.com/samoturk/mol2vec

In [ ]:
#importing required packages

from pathlib import Path
from tempfile import NamedTemporaryFile
import fileinput
import os
import rdkit
import pandas as pd
import numpy as np
import mol2vec
from mol2vec import features
from mol2vec import helpers
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import pkg_resources
#pkg_resources.require("gensim==3.8.3")  
import gensim
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import seaborn as sns
import matplotlib.pyplot as plt
import gensim.downloader
from rdkit import RDLogger   
RDLogger.DisableLog('rdApp.*') # turn off RDKit warning message 
import gc

In [ ]:
'''
These two functions are copied from the mol2Vec GitHub repository. 
It was needed to manually copy them and edit them in the notebook due to 
package version updates of several modules.

'''


def train_word2vec_model(infile_name, outfile_name=None, vector_size=100, window=10, min_count=3, n_jobs=1,
                         method='skip-gram', **kwargs):
    """Trains word2vec (Mol2vec, ProtVec) model on corpus file extracted from molecule/protein sequences.
    The corpus file is treated as LineSentence corpus (one sentence = one line, words separated by whitespaces)
    
    Parameters
    ----------
    infile_name : str
        Corpus file, e.g. proteins split in n-grams or compound identifier
    outfile_name : str
        Name of output file where word2vec model should be saved
    vector_size : int
        Number of dimensions of vector
    window : int
        Number of words considered as context
    min_count : int
        Number of occurrences a word should have to be considered in training
    n_jobs : int
        Number of cpu cores used for calculation
    method : str
        Method to use in model training. Options cbow and skip-gram, default: skip-gram)
    
    Returns
    -------
    word2vec.Word2Vec
    """
    if method.lower() == 'skip-gram':
        sg = 1
    elif method.lower() == 'cbow':
        sg = 0
    else:
        raise ValueError('skip-gram or cbow are only valid options')
  
    start = timeit.default_timer()
    corpus = word2vec.LineSentence(infile_name)
    model = word2vec.Word2Vec(corpus, size=vector_size, window=window, min_count=min_count, workers=n_jobs, sg=sg,
                              **kwargs)
    if outfile_name:
        model.save(outfile_name)
    
    stop = timeit.default_timer()
    print('Runtime: ', round((stop - start)/60, 2), ' minutes')
    return model


def sentences2vec(sentences, model, unseen=None):
    """Generate vectors for each sentence (list) in a list of sentences. Vector is simply a
    sum of vectors for individual words.
    
    Parameters
    ----------
    sentences : list, array
        List with sentences
    model : word2vec.Word2Vec
        Gensim word2vec model
    unseen : None, str
        Keyword for unseen words. If None, those words are skipped.
        https://stats.stackexchange.com/questions/163005/how-to-set-the-dictionary-for-text-analysis-using-neural-networks/163032#163032
    Returns
    -------
    np.array
    """
    keys = set(model.wv.vocab.keys())
    #keys = set(model.wv.index_to_key)
    vec = []
    if unseen:
        unseen_vec = model.wv.word_vec(unseen)

    for sentence in sentences:
        if unseen:
            vec.append(sum([model.wv.word_vec(y) if y in set(sentence) & keys
                       else unseen_vec for y in sentence]))
        else:
            vec.append(sum([model.wv.word_vec(y) for y in sentence 
                            if y in set(sentence) & keys]))
    return np.array(vec)

In [ ]:
'''
Three functions that assist in converting lists to strings and vice versa.
These are required in order to store vectors in csv files. 
'''


def str2float(string):
    split = list(string.split(','))
    floats_split = []
    for i in range(len(split)):
        floats = float(split[i])
        floats_split.append(floats)
    return floats_split

def stringToList(vectors):
    bracket_removed_mol2vec = []
    for i in range(len(vectors)):
        new_strings = vectors[i].replace('[', '')
        newer_strings = new_strings.replace(']', '')
        bracket_removed_mol2vec.append(newer_strings)

# Convert all vectors
    xList = []
    for i in range(len(bracket_removed_mol2vec)):
        float_vec = str2float(bracket_removed_mol2vec[i])
        xList.append(float_vec)
    
    return xList
    
def listToString(vectors):
    string_indices = []
    for i in range(len(vectors)):
        knn_string = ', '.join(str(k) for k in vectors[i])
        string_indices.append(knn_string)

    bracket_string_indices = []
    for i in range(len(string_indices)):
        bracket_string = '[' + string_indices[i] + ']'
        bracket_string_indices.append(bracket_string)
    
    return bracket_string_indices

In [ ]:
'''
Importing all SMILES strings in dataset
'''


path = os.path.join(os.getcwd(), 'fullDataset.csv')
full = pd.read_csv(path)
smile = list(full['smile'])

In [ ]:
'''
Writing all SMILES string into a .smi file
'''

fileName = 'allSmiles.smi'
filePath = os.path.join(os.getcwd(), fileName)
with open(filePath, 'w') as f:
    for smi in smile:
        f.write("{}\n".format(smi))

In [ ]:
'''
Generating substructure corpus that is used to train word2Vec model on the molecular dataset.
This takes around 20 minutes.

'''
CORPUSNAME = "mol2vec_corpus.dat"
RADIUS = 1
NJOBS = 32

fileName = 'allSmiles.smi'
filePath = os.path.join(os.getcwd(), fileName)

features.generate_corpus(
    filePath,
    CORPUSNAME,
    RADIUS,
    sentence_type="alt",
    n_jobs=NJOBS,
)

In [ ]:
'''
Training word2Vec model on substructure corpus. 
This trained model can then be used to create length 70 molecular feature vectors.
'''

modelName = 'mol2vec_model_final_70.pkl'
modelPath = os.path.join(os.getcwd(), modelName)

model = features.train_word2vec_model(corpusPath, modelPath, vector_size = 70, min_count=1, n_jobs=NJOBS)